# Notes

This jupyter notebook loads data based on R1_Activity_Plots.ipynb and puts it into a JSON-format data dictionary suitable for using with, e.g., frbpa (https://github.com/KshitijAggarwal/frbpa/tree/master/examples) which recreates the Rajwade et al. results from https://ui.adsabs.harvard.edu/abs/2020MNRAS.495.3551R/abstract

To use these packages anywhere with a module system, you can use the 'arecibo virtual environment' I have made which is at:

Raven:/u/cwalker/virtual_environments/install_arecibo_env.sh

To first setup the environment, run: 

> bash install_arecibo_env.sh

After first setup, use any time by running:

 > source /u/cwalker/virtual_environments/arecibo-env/bin/activate

# Imports

In [2]:
import json
import astropy

import numpy as np

from astropy.time import Time
from frbpa.utils import get_phase
from astropy.modeling import models, fitting
from frbpa.search import riptide_search, p4j_search

from astropy import units as u
from matplotlib import pyplot as plt
from scipy.signal import lombscargle as ls
from astropy.timeseries import TimeSeries as ts

2023-05-30 11:56:34,314 - _init_num_threads -numexpr.utils - INFO - Note: detected 144 virtual cores but NumExpr set to maximum of 64, check "NUMEXPR_MAX_THREADS" environment variable.
2023-05-30 11:56:34,318 - _init_num_threads -numexpr.utils - INFO - Note: NumExpr detected 144 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
2023-05-30 11:56:34,319 - _init_num_threads -numexpr.utils - INFO - NumExpr defaulting to 8 threads.


# Functions

In [3]:
def TallyBursts(ObservingTimes,ObservingDurations,BurstMJDs):
    """
    For papers which provide burst data but not observing information, we must find the observing 
    files where these bursts were stored on dop263.
    
    The bursts must then be tallied into their respective observations. This function does this.
    
    INPUTS:
    
    ObservingTimes     : [array of astropy mjd values] List of observation start times which
                         contained bursts. Must be calculated from observing data.
    ObservingDurations : [array of astropy seconds values] List of observation durations in u.s.
                         Must be calculated from observing data.
    BurstMJDs          : [array of floats] List of mjd values of the bursts observed. Must NOT
                         be an astropy time quantity.
    
    
    RETURNS:
    
    TalliedEvents : [array of ints of shape equal to list of observing times]
    """
    
    
    TalliedEvents = np.zeros_like(ObservingTimes)

    #print(TalliedEvents)
    for i in range(len(BurstMJDs)): #loop over marcote bursts
        for j in range(len(ObservingTimes)): #loop over observation start times
            if (Time(BurstMJDs[i],format='mjd') > ObservingTimes[j]) & (Time(BurstMJDs[i],format='mjd') < ObservingTimes[j] + ObservingDurations[j]): #if burst falls into obs
                #print(True)
                TalliedEvents[j]+=1 #tally burst
    if np.sum(TalliedEvents) != len(BurstMJDs):
        print('warning! Not all bursts are tallied. Code is wrong.')
    else:
        print('bursts tallied.')
        
    return TalliedEvents

# Load data

In [4]:
# Cruces et al. 2021
# https://ui.adsabs.harvard.edu/abs/2021MNRAS.500..448C/abstract

#key
Cruces_key = 'Cruces21a'

#reference
Cruces_ref = ['https://ui.adsabs.harvard.edu/abs/2021MNRAS.500..448C/abstract']

#telescope
Cruces_tel = 'Effelsberg'

#observing frequency
Cruces_freq = 1.36 #[GHz]

#burst information
Cruces_data = np.loadtxt('Cruces_Bursts.csv',dtype='str',skiprows=1,delimiter=',') #import all data
Cruces_mjds = np.array(Cruces_data[:,1],dtype=float) #select the mjd values

#observing information
Cruces_obs = np.loadtxt('Cruces_Obs.csv',dtype='str',skiprows=2,delimiter=',') #import all data
Cruces_Eff_obs = np.array([i for i in Cruces_obs if 'EFF' in i]) #extract only effelsberg observations
Cruces_Eff_times = Cruces_Eff_obs[:,1] #get the start utc of the observations
Cruces_Eff_times = Time(Cruces_Eff_times) #convert to astropy times
Cruces_Eff_events = np.array(Cruces_Eff_obs[:,-1],dtype=int) #get the number of events detected per observation
Cruces_Eff_durations = np.array(Cruces_Eff_obs[:,2],dtype=float)*u.s #get the duration of each observation in astropy s

In [5]:
# Hardy et al. 2017
# https://ui.adsabs.harvard.edu/abs/2017MNRAS.472.2800H/abstract

#key
Hardy_key = 'Hardy17a'

#reference
Hardy_ref = ['https://ui.adsabs.harvard.edu/abs/2017MNRAS.472.2800H/abstract']

#telescope
Hardy_tel = 'Effelsberg'

#observing frequency
Hardy_freq = 1.4 #[GHz]

#burst information
Hardy_data = np.loadtxt('Hardy_Bursts.csv',dtype='str',skiprows=2,delimiter=',') #import all data
Hardy_mjds = np.array(Hardy_data[:,0]) #select the mjd values
Hardy_mjds = np.array([i.split('(')[0] for i in Hardy_mjds],dtype=float) #for now, get rid of the rounded digits (may need to make precision better later)

#observing information
Hardy_obs = np.loadtxt('Hardy_Obs.csv',dtype='str',skiprows=2,delimiter=',') #import all data
Hardy_Eff_obs = Hardy_obs #all Hardy+17 observations were performed using the effelsberg
Hardy_Eff_times = Time(np.array(['{0} {1}'.format(i[0],i[1]) for i in Hardy_Eff_obs])) #convert to astropy times
print(Hardy_Eff_times)
Hardy_Eff_events = np.array(Hardy_obs[:,-1],dtype=int)
Hardy_Eff_Endtimes = Time(np.array(['{0} {1}'.format(i[0],i[2]) for i in Hardy_Eff_obs]))
Hardy_Eff_durations = (Hardy_Eff_Endtimes - Hardy_Eff_times).to(u.s) #get the duration of each observation in astropy s


['2017-01-16 16:14:20.000' '2017-01-19 16:05:40.000'
 '2017-01-25 16:22:00.000' '2017-02-16 14:09:40.000'
 '2017-02-19 14:28:40.000']


In [6]:
# Houben et al. 2019
# https://ui.adsabs.harvard.edu/abs/2019A%26A...623A..42H/abstract

#key
Houben_key = 'Houben19a'

#reference
Houben_ref = ['https://ui.adsabs.harvard.edu/abs/2019A%26A...623A..42H/abstract']

#telescope
Houben_tel = 'Effelsberg'

#observing frequency
Houben_freq = 1.4 #[GHz]

#burst information
Houben_data = np.loadtxt('Houben_Bursts.csv',dtype='str',skiprows=1,delimiter=',') #import all data
Houben_mjds = np.array(Houben_data[:,1],dtype=float) #select the mjd values

#observing information
Houben_obs = np.loadtxt('Houben_Obs.csv',dtype='str',skiprows=2,delimiter=',') #import all data
Houben_Eff_obs = np.array([i for i in Houben_obs if 'EFF' in i]) #extract only effelsberg observations
Houben_Eff_times = Time(np.array(['{0} {1}'.format(i[1],i[2]) for i in Houben_Eff_obs])) #convert to astropy times
Houben_Eff_events = np.array(Houben_Eff_obs[:,-1],dtype=int)
Houben_Eff_durations = np.array(Houben_Eff_obs[:,3],dtype=str) #get the duration of each observation
Houben_Eff_durations = [i.split(' ') for i in Houben_Eff_durations]
Houben_Eff_hours = np.array([i[0].split('h')[0] for i in Houben_Eff_durations],dtype=int)*u.hr
Houben_Eff_mins = np.array([i[1].split('m')[0] for i in Houben_Eff_durations],dtype=int)*u.min
Houben_Eff_durations = (Houben_Eff_hours+Houben_Eff_mins).to(u.s) #convert to astropy s

In [7]:
# ALFA bursts between Nov. 2012 and Jun. 2015
# burst info from Spitler et al. 2016:
# https://ui.adsabs.harvard.edu/abs/2016Natur.531..202S/abstract
# obs info from Scholtz et al. 2016:
# https://ui.adsabs.harvard.edu/abs/2016ApJ...833..177S/abstract

#key
ALFA_key = 'ALFA16a'

#reference
ALFA_ref = ['https://ui.adsabs.harvard.edu/abs/2016Natur.531..202S/abstract',
            'https://ui.adsabs.harvard.edu/abs/2016ApJ...833..177S/abstract']

#observing frequency
ALFA_freq = 1.4 #[GHz]

#telescope
ALFA_tel = 'Arecibo'

#burst information
ALFA_bursts = np.loadtxt('ALFA_Bursts.txt',dtype=str,skiprows=1,delimiter=' ') #load burst info
ALFA_mjds = np.array(ALFA_bursts[:,1],dtype=float) #get burst mjds

#observing information
ALFA_obs = np.loadtxt('ALFA_Obs.txt',dtype=str,skiprows=1,delimiter=',') #load observation information
ALFA_times = np.array(ALFA_obs[:,0],dtype='str') #load observation information
ALFA_times = Time(ALFA_times) #convert to astropy times
ALFA_events = np.array(ALFA_obs[:,3],dtype=int) #number of events detected per observation
ALFA_durations = np.array(ALFA_obs[:,2],dtype=int) #duration of each observation in seconds
ALFA_durations*=u.s

In [8]:
# non-detection observations (from within Jan 2016) from Scholtz et al 2016:
# https://ui.adsabs.harvard.edu/abs/2016ApJ...833..177S/abstract
# which were not also in Hewitt et al 2022:
# https://ui.adsabs.harvard.edu/abs/2021arXiv211111282H/abstract

#key
Scholtz16_NH_key = 'Scholtz16a'

#reference
Scholtz16_NH_ref = ['https://ui.adsabs.harvard.edu/abs/2016ApJ...833..177S/abstract',
                    'https://ui.adsabs.harvard.edu/abs/2021arXiv211111282H/abstract']
#observing frequency
Scholtz16_NH_freq = 1.4 #[GHz]

#telescope
Scholtz16_NH_tel = 'Arecibo'

#burst information
Scholtz16_NH_bursts = np.expand_dims(np.loadtxt('Scholtz16_NonHewitt_Bursts.txt',dtype=str,skiprows=1,delimiter=','),0)
Scholtz16_NH_mjds = np.array([]) #note: there were no bursts in these obs so this is placeholder

#observing information
Scholtz16_NH_obs = np.loadtxt('Scholtz16_NonHewitt_Obs.txt',dtype=str,delimiter=',',skiprows=1)
Scholtz16_NH_times = np.array(Scholtz16_NH_obs[:,0],dtype=str)#starting time of observation
Scholtz16_NH_times = Time(Scholtz16_NH_times)#.mjd #convert to astropy times
Scholtz16_NH_events = np.array(Scholtz16_NH_obs[:,3],dtype=int) #all scholtz+17 Arecibo bursts were in a single observation
Scholtz16_NH_durations=np.array(Scholtz16_NH_obs[:,2],dtype=int) #get duration of observation in seconds
Scholtz16_NH_durations*=u.s #convert to astropy units

In [9]:
# Hewitt et al 2022 bursts between Nov. 2015 and Oct. 2016
# https://ui.adsabs.harvard.edu/abs/2021arXiv211111282H/abstract

#key
Hewitt_key = 'Hewitt22a'

#reference
Hewitt_ref = ['https://ui.adsabs.harvard.edu/abs/2021arXiv211111282H/abstract']

#observing frequency
Hewitt_freq = 1.4 #[GHz]

#telescope
Hewitt_tel = 'Arecibo'

#burst information
Hewitt_bursts = np.loadtxt('Hewitt_Bursts.txt',dtype='str',delimiter='&') #import Hewitt bursts
Hewitt_mjds = np.array(Hewitt_bursts[:,1],dtype=float) #mjds of the bursts

#observing information
Hewitt_obs = np.loadtxt('Hewitt_Obs.txt',dtype=str,delimiter='&')#Hewitt observing times
Hewitt_times = np.array(Hewitt_obs[:,0],dtype=float) #starting times of observations
Hewitt_times = Time(Hewitt_times,format='mjd')#.mjd #convert to astropy times
Hewitt_events = np.array(Hewitt_obs[:,2],dtype=int) #get number of events detected per observation
Hewitt_durations = np.array(Hewitt_obs[:,1],dtype=float) #get duration of observations in seconds
Hewitt_durations = Hewitt_durations*u.s #convert to astropy seconds

In [10]:
# Scholtz et al 2017 bursts from Jan. 2017
# https://ui.adsabs.harvard.edu/abs/2017ApJ...846...80S/abstract

#key
Scholtz17_key = 'Scholtz17a'

#reference
Scholtz17_ref = ['https://ui.adsabs.harvard.edu/abs/2017ApJ...846...80S/abstract']

#observing frequency
Scholtz17_freq = 1.4 #[GHz]

#telescope
Scholtz17_tel = 'Arecibo'

#burst information
Scholtz17_bursts = np.loadtxt('Scholtz17_Bursts.txt',dtype=str,skiprows=1,delimiter=',')
Scholtz17_mjds = np.array(Scholtz17_bursts[:,1],dtype=float) #mjds of bursts

#observing information
Scholtz17_obs = np.loadtxt('Scholtz17_Obs.txt',dtype=str,delimiter=',',skiprows=1)
Scholtz17_obs = np.expand_dims(Scholtz17_obs,0) #make a 2d array
Scholtz17_times = np.array(Scholtz17_obs[:,2],dtype=str)#starting time of observation
Scholtz17_times = Time(Scholtz17_times)#.mjd #convert to astropy times
Scholtz17_events = np.array([len(Scholtz17_mjds)]) #all scholtz+17 Arecibo bursts were in a single observation
Scholtz17_durations=np.array(Scholtz17_obs[:,4],dtype=int) #get duration of observation in seconds
Scholtz17_durations*=u.s #convert to astropy units

In [11]:
# Jahns et al 2022 bursts from Oct. 2018 - Nov. 2018
# https://ui.adsabs.harvard.edu/abs/2022arXiv220205705J/abstract

#key
Jahns_key = 'Jahns22a'

#reference
Jahns_ref = ['https://ui.adsabs.harvard.edu/abs/2022arXiv220205705J/abstract']

#observing frequency
Jahns_freq = 1.4 #[GHz]

#telescope
Jahns_tel = 'Arecibo'

#burst information
Jahns_bursts = np.loadtxt('Jahns_Bursts.txt',dtype='str',delimiter=',',skiprows=1) #load burst data
Jahns_mjds = np.array(Jahns_bursts[:,2],dtype=float) #get burst mjds

#observing information
Jahns_obs = np.loadtxt('Jahns_Obs.txt',dtype=str,delimiter='&')
Jahns_times = np.array(Jahns_obs[:,2],dtype=float) #starting times of observations
Jahns_times = Time(Jahns_times,format='mjd')#.mjd #convert to astropy times
Jahns_events = np.array(Jahns_obs[:,4],dtype=int) #get number of events per detected observation
Jahns_durations = np.array(Jahns_obs[:,3],dtype=float) #get duration of observations in seconds
Jahns_durations*=u.s #convert to astropy seconds

In [12]:
# Michilli et al 2018 C-band bursts from Nov. 2016 - May 2017
# https://ui.adsabs.harvard.edu/abs/2018Natur.553..182M/abstract

#key
Michilli_key = 'Michilli18a'

#reference
Michilli_ref = ['https://ui.adsabs.harvard.edu/abs/2018Natur.553..182M/abstract']

#observing frequency
Michilli_freq = 4.5 #[GHz]

#telescope
Michilli_tel = 'Arecibo'

#burst information
Michilli_bursts = np.loadtxt('Michilli_Bursts.txt',dtype='str',delimiter=' ') #import Michilli bursts
Michilli_mjds = np.array(Michilli_bursts[:,1],dtype=float) #mjds of the bursts

#observing information
Michilli_obs = np.loadtxt('Michilli_Obs.txt',dtype='str', delimiter = ' ') #Michilli observing times
Michilli_times = np.array(Michilli_obs[:,0],dtype=float) #starting time of observations
Michilli_times = Time(Michilli_times,format='mjd')#.mjd #convert to astropy times
Michilli_events = np.array(Michilli_obs[:,2],dtype=int) #get number of events per observation
Michilli_durations = np.array(Michilli_obs[:,1],dtype=float) #get duration of observations in seconds
Michilli_durations*=u.s #cconvert to astropy seconds

In [13]:
# Marcote et al 2016 bursts
# Note: there was not enough observing information in the literature. It had
# to be extracted manually from the observing session data stored on dop263
# https://ui.adsabs.harvard.edu/abs/2017ApJ...834L...8M/abstract

#key
Marcote_key = 'Marcote16a'

#reference
Marcote_ref = ['https://ui.adsabs.harvard.edu/abs/2017ApJ...834L...8M/abstract',
               'dop263:/data/FRB121102/pipeline_products/puppi_57651_C0531+33_0004']

#observing frequency
Marcote_freq = 1.7 #[GHz]

#telescope
Marcote_tel = 'Arecibo'

#burst information
Marcote_bursts = np.loadtxt('Marcote_Bursts.txt',dtype='str',delimiter=',')#Load all burst data
Marcote_mjds = Time(np.array(Marcote_bursts)[:,0]).mjd

#observing information
#data for this date is stored in: dop263:/data/FRB121102/pipeline_products/puppi_57651_C0531+33_0004
Marcote_times = np.array([57651.38309027778],dtype=float) #obs start times calculated from observing data
Marcote_times = Time(Marcote_times,format='mjd') #convert to astropy times
Marcote_durations = np.array([6556.72475648],dtype=float) # [s] calculated from observing data
Marcote_durations*=u.s
#tally bursts into their respective observations
Marcote_events = TallyBursts(Marcote_times,Marcote_durations,Marcote_mjds) #number of events per detected obs


bursts tallied.


In [14]:
# MAGIC collaboration 2017 bursts
# Note: there was not enough observing informaation in the literature. It had
# to be extracted manually from observing session dataa stored on dop263
# https://ui.adsabs.harvard.edu/abs/2018MNRAS.481.2479M/abstract

#key
MAGIC_key = 'MAGIC17a'

#reference
MAGIC_ref = ['https://ui.adsabs.harvard.edu/abs/2018MNRAS.481.2479M/abstract',
             'dop263:/data/FRB121102/pipeline_products/puppi_57806_C0531+33_0144',
             'dop263:/data/FRB121102/pipeline_products/puppi_57807_frb_0165',
             'dop263:/data/FRB121102/pipeline_products/puppi_57807_frb_0165']

#observing frequency
MAGIC_freq = 1.4 #[GHz]

#telescope
MAGIC_tel = 'Arecibo'

#burst information
MAGIC_bursts = np.loadtxt('MAGIC_Bursts.txt',dtype='str') #load all burst data
MAGIC_mjds = np.array(MAGIC_bursts[:,0],dtype=float) #get burst mjds

#observing information
MAGIC_times = np.array([57799.971979166665,57806.962175925924,57807.970925925925,57814.93099537037],dtype=float)
MAGIC_times = Time(MAGIC_times,format='mjd')
MAGIC_durations = np.array([6806.68430336,2700.2929152,5054.9751808,2931.3990656],dtype=float)
MAGIC_durations*=u.s
MAGIC_events = TallyBursts(MAGIC_times,MAGIC_durations,MAGIC_mjds)

bursts tallied.


In [15]:
# Spitler et al 2018 C-band Effelsberg bursts
# https://ui.adsabs.harvard.edu/abs/2018ApJ...863..150S/abstract

#key
Spitler_C_key = 'Spitler18a'

#reference
Spitler_C_ref = ['https://ui.adsabs.harvard.edu/abs/2018ApJ...863..150S/abstract']

#observing frequency
Spitler_C_freq = 4.85 #[GHz]

#telescope
Spitler_C_tel = 'Effelsberg'

#burst information
Spitler_C_bursts = np.loadtxt('Spitler_CBand_Eff_Bursts.csv',dtype=str,delimiter=',',skiprows=1) #load all burst data
Spitler_C_mjds = np.array(Spitler_C_bursts[:,5],dtype=float) #get burst mjds

#observing information
Spitler_C_obs = np.loadtxt('Spitler_CBand_Eff_Obs.csv',dtype=str,delimiter=',',skiprows=1)
Spitler_C_times = np.array(Spitler_C_obs[:,2],dtype=float)#get start times of observations
Spitler_C_times = Time(Spitler_C_times,format='mjd')#convert to astropy mjds
Spitler_C_durations = np.array(Spitler_C_obs[:,3],dtype=float) #durations of observations in seconds
Spitler_C_durations = Spitler_C_durations*u.s #convert to astropy seconds
Spitler_C_events = TallyBursts(Spitler_C_times,Spitler_C_durations,Spitler_C_mjds)

bursts tallied.


In [16]:
"""
# Gaijar et al 2018 GBT bursts
# Note: paper doesn't state start time of the observation.
# however it does say observation was 6 hours long, first hour was
# spent calibrating, and all bursts were detected in the second hour
# with no detections in the last four hours. We therefore ESTIMATE
# the observing start time to be just before the first burst.
#https://ui.adsabs.harvard.edu/abs/2018ApJ...863....2G/abstract

#key
Gaijar_key = 'Gaijar18a'

#reference
Gaijar_ref = ['https://ui.adsabs.harvard.edu/abs/2018ApJ...863....2G/abstract']

#observing frequency
Gaijar_freq = 6 #[GHz]

#telescope
Gaijar_tel = 'GBT'

#burst information
Gaijar_bursts = np.loadtxt('Gaijar_GBT_Bursts.txt',dtype=str,delimiter=',',skiprows=1) #load all burst data
Gaijar_mjds = np.array(Gaijar_bursts[:,1],dtype=float)+57991 #get burst mjds
print('Gaijar burst times at: {0}'.format(Time(Gaijar_mjds,format='mjd').datetime))

#observing information
#ESTIMATE(!) observing start time to be one min before first burst. See above note.
Gaijar_times=np.array([np.amin(Gaijar_mjds)],dtype=float) 
Gaijar_times = Time(Gaijar_times,format='mjd')-(1*u.min)
Gaijar_durations = np.array([18000],dtype=float) #note: observation was 6 hours, but only 5 on source
Gaijar_durations*=u.s
Gaijar_events = TallyBursts(Gaijar_times,Gaijar_durations,Gaijar_mjds)


print(Gaijar_times,Gaijar_durations)
"""

"\n# Gaijar et al 2018 GBT bursts\n# Note: paper doesn't state start time of the observation.\n# however it does say observation was 6 hours long, first hour was\n# spent calibrating, and all bursts were detected in the second hour\n# with no detections in the last four hours. We therefore ESTIMATE\n# the observing start time to be just before the first burst.\n#https://ui.adsabs.harvard.edu/abs/2018ApJ...863....2G/abstract\n\n#key\nGaijar_key = 'Gaijar18a'\n\n#reference\nGaijar_ref = ['https://ui.adsabs.harvard.edu/abs/2018ApJ...863....2G/abstract']\n\n#observing frequency\nGaijar_freq = 6 #[GHz]\n\n#telescope\nGaijar_tel = 'GBT'\n\n#burst information\nGaijar_bursts = np.loadtxt('Gaijar_GBT_Bursts.txt',dtype=str,delimiter=',',skiprows=1) #load all burst data\nGaijar_mjds = np.array(Gaijar_bursts[:,1],dtype=float)+57991 #get burst mjds\nprint('Gaijar burst times at: {0}'.format(Time(Gaijar_mjds,format='mjd').datetime))\n\n#observing information\n#ESTIMATE(!) observing start time to be 

In [46]:
#Zhang et al 2018 GBT bursts
#paper: https://iopscience.iop.org/article/10.3847/1538-4357/aadf31

#Note: this is a re-analysis of Gaijar et al. 2018 (https://ui.adsabs.harvard.edu/abs/2018ApJ...863....2G/abstract)
#which includes those bursts, but also found more bursts via neural networks.
#so this paper can replace Gaijar2018, which is why the above is now commented out.

#Note: using the Table 2 caption, we can calculate the start time of the observation.
#Also note: according to paper, the observation was 6 hours, but only 300 minutes (i.e. 5 hours) were on-source.

#key
Zhang_key = 'Zhang18a'

#reference
Zhang_ref = ['https://ui.adsabs.harvard.edu/abs/2018ApJ...866..149Z/abstract']

#observing frequency
Zhang_freq = 6

#telescope
Zhang_tel = 'GBT'

#burst information
Zhang_bursts = np.loadtxt('Zhang_GBT_Bursts.txt',dtype=str,delimiter=',',skiprows=1) #load all burst data
Zhang_mjds = np.array(Zhang_bursts[:,1],dtype=float)+57991 #get burst mjds
#print('Zhang burst times at: {0}'.format(Time(Zhang_mjds,format='mjd').datetime))

#observing information
#note: only one observation, and first burst was 16.2278s after it started
Zhang_times = Time(np.array([np.amin(Zhang_mjds)]),format='mjd')-16.2278*u.s
print(Zhang_times.mjd)
#Zhang_times=np.array([np.amin(Zhang_mjds)],dtype=float) 
#Zhang_times = Time(Zhang_times,format='mjd')-(1*u.min)
Zhang_durations = np.array([18000],dtype=float) #note: observation was 6 hours, but only 5 on source
Zhang_durations*=u.s
Zhang_events = TallyBursts(Zhang_times,Zhang_durations,Zhang_mjds)


[57991.40971622]
bursts tallied.


In [27]:
# Rajwade et al 2020 Lovell bursts
#Note: the data comes from the paper's supplementary material
#Note: Observation 86 in the lovell obs table (time in mjd : 58459.22533564 - 58459.267233)
#does not provide information on bursts detected. I have deleted it for now.
#Could contact authors to find out why it is blank.
#https://ui.adsabs.harvard.edu/abs/2020MNRAS.495.3551R/abstract

#key
Rajwade_key = 'Rajwade20a'

#reference
Rajwade_ref = ['https://ui.adsabs.harvard.edu/abs/2020MNRAS.495.3551R/abstract']

#observing frequency
Rajwade_freq = 1.4 #[GHz]

#telescope
Rajwade_tel = 'Lovell'

#burst information
Rajwade_bursts = np.loadtxt('Rajwade_Lovell_Bursts.csv',dtype=str,delimiter=',',skiprows=1) #load all burst data
Rajwade_mjds = np.array(Rajwade_bursts[:,1],dtype=float) #get burst mjds
#print('Rajwade burst times at: {0}'.format(Time(Rajwade_mjds,format='mjd').datetime))

#observing information
Rajwade_obs = np.loadtxt('Rajwade_Lovell_Obs.csv',dtype='str', delimiter = ',',skiprows=1) #Rajwade observing times
Rajwade_times = np.array(Rajwade_obs[:,1],dtype=float) #starting time of observations
Rajwade_times = Time(Rajwade_times,format='mjd')#.mjd #convert to astropy times
Rajwade_events = np.array(Rajwade_obs[:,3],dtype=int) #get number of events per observation
Rajwade_durations = (Time(np.array(Rajwade_obs[:,2],dtype=float),format='mjd')-Rajwade_times).to('s') #get duration of observations in astropy seconds

print(Rajwade_times,Rajwade_durations)

[57363.9001968 57365.0012963 57371.1676968 57379.2055208 57389.0533796
 57402.0379051 57407.1136111 57428.8283333 57429.0005556 57430.0000463
 57463.5381713 57464.5426389 57473.8181134 57503.5778819 57522.5927199
 57542.5871991 57600.6521759 57601.464838  57607.2050463 57611.3930208
 57625.2286111 57636.4117593 57660.1332523 57731.9311806 57732.9692361
 57733.0000231 57734.0149074 57735.0182755 57736.0044213 57737.0006134
 57738.0987269 57740.0132523 57741.156713  57742.1322222 57743.1898958
 57749.1250694 57750.1050347 57754.0425694 57756.1038773 57757.0570255
 57758.1246296 57759.1666551 57760.991794  57761.0111574 57762.1394676
 57763.9360417 57764.9411111 57765.9590856 57766.0000231 57767.1370139
 57768.1482755 57769.1381134 57770.1217361 57771.0228472 57777.1605093
 57779.940706  57781.7416204 57785.0630556 57787.7983565 57789.0112037
 57790.8103472 57791.9167593 57793.023287  57797.8964236 57798.8532176
 57800.1026968 57801.1211227 57805.9416667 57806.9484375 57818.9738426
 57819

In [28]:
# Oostrum et al., 2017
#Note: Atel, single observation of 2 hours at 1.4GHz
#https://www.astronomerstelegram.org/?read=10693

#key
Oostrum2017_key = 'Oostrum17a'
Oostrum2017_ref = ['https://www.astronomerstelegram.org/?read=10693']

#observing frequency
Oostrum2017_freq = 1.4 #GHz

#telescope
Oostrum2017_tel = 'WSRT'

#burst information
Oostrum2017_mjds = np.array([57996.2656372]) #burst mjds

#observing information #2017-08-31 06:23:37
Oostrum2017_times = Time(Time(['2017-08-31 06:23:37']).mjd,format='mjd') #starting time of observations
Oostrum2017_events = np.array([1]) #number of events per observation
Oostrum2017_durations = (np.array([2])*u.hr).to('s')

print(Oostrum2017_times,Oostrum2017_durations)


[57996.26640046] [7200.] s


In [29]:
# Caleb et al., 2020 (MeerKAT data)
# First MeerKAT observations on September 10th detected 11 bursts
# Follow-up observations with MeerKAT, Sardinia, Nancay and Effelsberg did not detect bursts
#Note: the observations by MeerKAT on 2019-10-06, 2019-10-08 do not provide enough 
#observing info. Should contact the authors. However, they were simultaneous with
#Eff, Nancay, so I will have some of the information.
#Note: the first burst mjd actually occurs BEFORE the start of the observation
#according to the provided start time of 03:43 UT. Check this with author.
#https://ui.adsabs.harvard.edu/abs/2020MNRAS.496.4565C/abstract

CalebMeerKAT20a_key = 'CalebMeerKAT20a'
CalebMeerKAT20a_ref = 'https://ui.adsabs.harvard.edu/abs/2020MNRAS.496.4565C/abstract'

#Observing frequency
CalebMeerKAT20a_freq = 1.4 #GHz

#telescope
CalebMeerKAT20a_tel = 'MeerKAT'

#burst information
CalebMeerKAT20a_bursts = np.loadtxt('Caleb_MeerKAT_bursts.csv',dtype=str,delimiter=',',skiprows=2) #load all burst data
CalebMeerKAT20a_mjds = np.array(CalebMeerKAT20a_bursts[:,1],dtype='float') #burst mjds

#observing information
CalebMeerKAT20a_obs = np.loadtxt('Caleb_MeerKAT_Obs.csv',dtype='str', delimiter = ',',skiprows=1) #Rajwade observing times
CalebMeerKAT20a_times = Time(Time(np.array([CalebMeerKAT20a_obs[0]]),scale='utc').mjd,format='mjd') #starting time of observations
CalebMeerKAT20a_events = np.array([CalebMeerKAT20a_obs[2]],dtype='int') #number of events per observation
CalebMeerKAT20a_durations = (np.array([CalebMeerKAT20a_obs[1]],dtype='float')*u.hr).to('s') #duration of observations
print(CalebMeerKAT20a_mjds)
print(CalebMeerKAT20a_times)
print(CalebMeerKAT20a_events)
print(CalebMeerKAT20a_durations)
print(Time(np.array(CalebMeerKAT20a_obs[0]),scale='utc'))

[58736.14855 58736.16565 58736.16566 58736.16694 58736.18484 58736.2002
 58736.21155 58736.23518 58736.24062 58736.24786 58736.25422]
[58736.15486111]
[11]
[10800.] s
2019-09-10 03:43:00.000


In [30]:
# Caleb et al., 2020 (Sardinia data)
# Sardinia follow-up observations with no detected bursts

#https://ui.adsabs.harvard.edu/abs/2020MNRAS.496.4565C/abstract

CalebSardinia20a_key = 'CalebSardinia20a'
CalebSardinia20a_ref = 'https://ui.adsabs.harvard.edu/abs/2020MNRAS.496.4565C/abstract'

#Observing frequency
CalebSardinia20a_freq = 0.336 #GHz

#telescope
CalebSardinia20a_tel = 'Sardinia'

#burst information
#CalebMeerKAT20a_bursts = np.loadtxt('Caleb_MeerKAT_bursts.csv',dtype=str,delimiter=',',skiprows=2) #load all burst data
CalebSardinia20a_mjds = np.array([]) #burst mjds

#observing information
CalebSardinia20a_obs = np.loadtxt('Caleb_Sardinia_Obs.csv',dtype='str', delimiter = ',',skiprows=1) #Rajwade observing times
CalebSardinia20a_times = Time(Time(np.array([CalebSardinia20a_obs[0]]),scale='utc').mjd,format='mjd') #starting time of observations
CalebSardinia20a_events = np.array([CalebSardinia20a_obs[2]],dtype='int') #number of events per observation
CalebSardinia20a_durations = (np.array([CalebSardinia20a_obs[1]],dtype='float')*u.min).to('s') #duration of observations
print(CalebSardinia20a_mjds)
print(CalebSardinia20a_times)
print(CalebSardinia20a_events)
print(CalebSardinia20a_durations)
print(Time(np.array(CalebSardinia20a_obs[0]),scale='utc'))

[]
[58762.07340278]
[0]
[11700.] s
2019-10-06 01:45:42.000


In [58]:
# Caleb et al., 2020 (Effelsberg data)
# Effelsberg follow-up observations with no detected bursts

#https://ui.adsabs.harvard.edu/abs/2020MNRAS.496.4565C/abstract

CalebEffelsberg20a_key = 'CalebEffelsberg20a'
CalebEffelsberg20a_ref = 'https://ui.adsabs.harvard.edu/abs/2020MNRAS.496.4565C/abstract'

#Observing frequency
CalebEffelsberg20a_freq = 5 #GHz

#telescope
CalebEffelsberg20a_tel = 'Effelsberg'

#burst information
#CalebMeerKAT20a_bursts = np.loadtxt('Caleb_MeerKAT_bursts.csv',dtype=str,delimiter=',',skiprows=2) #load all burst data
CalebEffelsberg20a_mjds = np.array([]) #burst mjds

#observing information
CalebEffelsberg20a_obs = np.loadtxt('Caleb_Effelsberg_Obs.csv',dtype='str', delimiter = ',',skiprows=1) #Rajwade observing times
CalebEffelsberg20a_times = Time(Time(np.array(CalebEffelsberg20a_obs[:,0]),scale='utc').mjd,format='mjd') #starting time of observations
CalebEffelsberg20a_events = np.array(CalebEffelsberg20a_obs[:,2],dtype='int') #number of events per observation
CalebEffelsberg20a_durations = (np.array(CalebEffelsberg20a_obs[:,1],dtype='float')*u.min).to('s') #duration of observations
print('mjds',CalebEffelsberg20a_mjds)
print('times',CalebEffelsberg20a_times)
print('events',CalebEffelsberg20a_events)
print('durations',CalebEffelsberg20a_durations)
print(Time(np.array(CalebEffelsberg20a_obs[:,0]),scale='utc'))

mjds []
times [58762.18402778 58764.07291667]
events [0 0]
durations [14400. 21600.] s
['2019-10-06 04:25:00.000' '2019-10-08 01:45:00.000']


In [32]:
# Caleb et al., 2020 (Nancay data)
# Sardinia simultaneous observations with MeerKAT
# Some bursts were simultaneously detected with MeerKAT
#on Sep. 10th

#https://ui.adsabs.harvard.edu/abs/2020MNRAS.496.4565C/abstract

CalebNancay20a_key = 'CalebNancay20a'
CalebNancay20a_ref = 'https://ui.adsabs.harvard.edu/abs/2020MNRAS.496.4565C/abstract'

#Observing frequency
CalebNancay20a_freq = 1.486 #GHz

#telescope
CalebNancay20a_tel = 'Nancay'

#burst information
CalebNancay20a_bursts = np.loadtxt('Caleb_Nancay_bursts.csv',dtype=str,delimiter=',',skiprows=2) #load all burst data
CalebNancay20a_mjds = np.array(CalebNancay20a_bursts[:,1],dtype='float') #burst mjds

#observing information
CalebNancay20a_obs = np.loadtxt('Caleb_Nancay_Obs.csv',dtype='str', delimiter = ',',skiprows=1) #Nancay observing times
CalebNancay20a_times = Time(Time(np.array(CalebNancay20a_obs[:,0]),scale='utc').mjd,format='mjd') #starting time of observations
CalebNancay20a_events = np.array(CalebNancay20a_obs[:,2],dtype='int') #number of events per observation
CalebNancay20a_durations = (np.array(CalebNancay20a_obs[:,1],dtype='float')*u.min).to('s') #duration of observations
print(CalebNancay20a_mjds)
print(CalebNancay20a_times)
print(CalebNancay20a_events)
print(CalebNancay20a_durations)
print(Time(np.array(CalebNancay20a_obs[:,0]),scale='utc'))

[58736.23518 58736.24062 58736.24786 58736.25422]
[58736.22893519 58762.1603588 ]
[4 0]
[4500. 4260.] s
['2019-09-10 05:29:40.000' '2019-10-06 03:50:55.000']


##### Need more data for

In [22]:
#Load Hilmarsson Arecibo data not in Michilli (4GHz)
Hilmarsson_bursts = np.loadtxt('Hilmarsson_New_Arecibo_Bursts.txt',dtype='str') #load all burst data
Hilmarsson_mjds = np.array(Hilmarsson_bursts[:,1],dtype=float) #get burst mjds
#Hilmarsson_obs = 
#print(Hilmarsson_mjds)
#print(Time(Hilmarsson_mjds,format='mjd').datetime)

In [23]:
#Load Chatterjee VLA data (3GHz)
#https://ui.adsabs.harvard.edu/abs/2017Natur.541...58C/abstract
#note: need observing duration info and have contacted author
ChatterjeeVLA17_bursts = np.loadtxt('Chatterjee_VLA_bursts.csv',dtype='str', delimiter = ',',skiprows=1)
ChatterjeeVLA17_mjds = np.array(ChatterjeeVLA17_bursts[:,1],dtype=float) #get burst mjds
print(ChatterjeeVLA17_mjds)

[57623.74402686 57633.67986367 57633.69515938 57638.49937435
 57643.45730263 57645.42958602 57646.4660065  57648.4369149
 57649.45175697]


# Create dictionaries

## Burst dictionaries

### L-band


In [33]:
#initialise data dictionaries
L_bursts_dict = {} #dictionary to hold L-band bursts
L_refs_dict = {} #dictionary to hold corresponding references
L_tels_dict = {} #dictionary to hold corresponding telescopes
L_freqs_dict = {} #dictionary to hold corresponding frequencies
L_obs_tels_dict = {} #dictionary to hold corresponding telescopes
L_obs_startmjds_dict = {} #dict to hold corresponding observation start mjds
L_obs_durations_dict = {} #dict to hold corresponding observation durations [s]
L_events_per_obs_dict = {} #dict to hold corresponding detections per individual obs

In [34]:
#add data

#burst mjds
L_bursts_dict[Cruces_key] = Cruces_mjds.tolist()
L_bursts_dict[Hardy_key] = Hardy_mjds.tolist()
L_bursts_dict[Houben_key] = Houben_mjds.tolist()
L_bursts_dict[ALFA_key] = ALFA_mjds.tolist()
L_bursts_dict[Scholtz16_NH_key] = Scholtz16_NH_mjds.tolist()
L_bursts_dict[Hewitt_key] = Hewitt_mjds.tolist()
L_bursts_dict[Scholtz17_key] = Scholtz17_mjds.tolist()
L_bursts_dict[Jahns_key] = Jahns_mjds.tolist()
L_bursts_dict[Marcote_key] = Marcote_mjds.tolist()
L_bursts_dict[MAGIC_key] = MAGIC_mjds.tolist()
L_bursts_dict[Rajwade_key] = Rajwade_mjds.tolist()
L_bursts_dict[Oostrum2017_key] = Oostrum2017_mjds.tolist()
L_bursts_dict[CalebMeerKAT20a_key] = CalebMeerKAT20a_mjds.tolist()
L_bursts_dict[CalebSardinia20a_key] = CalebSardinia20a_mjds.tolist()
L_bursts_dict[CalebNancay20a_key] = CalebNancay20a_mjds.tolist()

#references
L_refs_dict[Cruces_key] = Cruces_ref
L_refs_dict[Hardy_key] = Hardy_ref
L_refs_dict[Houben_key] = Houben_ref
L_refs_dict[ALFA_key] = ALFA_ref
L_refs_dict[Scholtz16_NH_key] = Scholtz16_NH_ref
L_refs_dict[Hewitt_key] = Hewitt_ref
L_refs_dict[Scholtz17_key] = Scholtz17_ref
L_refs_dict[Jahns_key] = Jahns_ref
L_refs_dict[Marcote_key] = Marcote_ref
L_refs_dict[MAGIC_key] = MAGIC_ref
L_refs_dict[Rajwade_key] = Rajwade_ref
L_refs_dict[Oostrum2017_key] = Oostrum2017_ref
L_refs_dict[CalebMeerKAT20a_key] = CalebMeerKAT20a_ref
L_refs_dict[CalebSardinia20a_key] = CalebSardinia20a_ref
L_refs_dict[CalebNancay20a_key] = CalebNancay20a_ref

#telescopes
L_tels_dict[Cruces_key] = Cruces_tel
L_tels_dict[Hardy_key] = Hardy_tel
L_tels_dict[Houben_key] = Houben_tel
L_tels_dict[ALFA_key] = ALFA_tel
L_tels_dict[Scholtz16_NH_key] = Scholtz16_NH_tel
L_tels_dict[Hewitt_key] = Hewitt_tel
L_tels_dict[Scholtz17_key] = Scholtz17_tel
L_tels_dict[Jahns_key] = Jahns_tel
L_tels_dict[Marcote_key] = Marcote_tel
L_tels_dict[MAGIC_key] = MAGIC_tel
L_tels_dict[Rajwade_key] = Rajwade_tel
L_tels_dict[Oostrum2017_key] = Oostrum2017_tel
L_tels_dict[CalebMeerKAT20a_key] = CalebMeerKAT20a_tel
L_tels_dict[CalebSardinia20a_key] = CalebSardinia20a_tel
L_tels_dict[CalebNancay20a_key] = CalebNancay20a_tel

#observing frequencies
L_freqs_dict[Cruces_key] = Cruces_freq
L_freqs_dict[Hardy_key] = Hardy_freq
L_freqs_dict[Houben_key] = Houben_freq
L_freqs_dict[ALFA_key] = ALFA_freq
L_freqs_dict[Scholtz16_NH_key] = Scholtz16_NH_freq
L_freqs_dict[Hewitt_key] = Hewitt_freq
L_freqs_dict[Scholtz17_key] = Scholtz17_freq
L_freqs_dict[Jahns_key] = Jahns_freq
L_freqs_dict[Marcote_key] = Marcote_freq
L_freqs_dict[MAGIC_key] = MAGIC_freq
L_freqs_dict[Rajwade_key] = Rajwade_freq
L_freqs_dict[Oostrum2017_key] = Oostrum2017_freq
L_freqs_dict[CalebMeerKAT20a_key] = CalebMeerKAT20a_freq
L_freqs_dict[CalebSardinia20a_key] = CalebSardinia20a_freq
L_freqs_dict[CalebNancay20a_key] = CalebNancay20a_freq

#start mjds
L_obs_startmjds_dict[Cruces_key] = Cruces_Eff_times.mjd.tolist()
L_obs_startmjds_dict[Hardy_key] = Hardy_Eff_times.mjd.tolist()
L_obs_startmjds_dict[Houben_key] = Houben_Eff_times.mjd.tolist()
L_obs_startmjds_dict[ALFA_key] = ALFA_times.mjd.tolist()
L_obs_startmjds_dict[Scholtz16_NH_key] = Scholtz16_NH_times.mjd.tolist()
L_obs_startmjds_dict[Hewitt_key] = Hewitt_times.mjd.tolist()
L_obs_startmjds_dict[Scholtz17_key] = Scholtz17_times.mjd.tolist()
L_obs_startmjds_dict[Jahns_key] = Jahns_times.mjd.tolist()
L_obs_startmjds_dict[Marcote_key] = Marcote_times.mjd.tolist()
L_obs_startmjds_dict[MAGIC_key] = MAGIC_times.mjd.tolist()
L_obs_startmjds_dict[Rajwade_key] = Rajwade_times.mjd.tolist()
L_obs_startmjds_dict[Oostrum2017_key] = Oostrum2017_times.mjd.tolist()
L_obs_startmjds_dict[CalebMeerKAT20a_key] = CalebMeerKAT20a_times.mjd.tolist()
L_obs_startmjds_dict[CalebSardinia20a_key] = CalebSardinia20a_times.mjd.tolist()
L_obs_startmjds_dict[CalebNancay20a_key] = CalebNancay20a_times.mjd.tolist()

#durations
L_obs_durations_dict[Cruces_key] = Cruces_Eff_durations.value.tolist()
L_obs_durations_dict[Hardy_key] = Hardy_Eff_durations.value.tolist()
L_obs_durations_dict[Houben_key] = Houben_Eff_durations.value.tolist()
L_obs_durations_dict[ALFA_key] = ALFA_durations.value.tolist()
L_obs_durations_dict[Scholtz16_NH_key] = Scholtz16_NH_durations.value.tolist()
L_obs_durations_dict[Hewitt_key] = Hewitt_durations.value.tolist()
L_obs_durations_dict[Scholtz17_key] = Scholtz17_durations.value.tolist()
L_obs_durations_dict[Jahns_key] = Jahns_durations.value.tolist()
L_obs_durations_dict[Marcote_key] = Marcote_durations.value.tolist()
L_obs_durations_dict[MAGIC_key] = MAGIC_durations.value.tolist()
L_obs_durations_dict[Rajwade_key] = Rajwade_durations.value.tolist()
L_obs_durations_dict[Oostrum2017_key] = Oostrum2017_durations.value.tolist()
L_obs_durations_dict[CalebMeerKAT20a_key] = CalebMeerKAT20a_durations.value.tolist()
L_obs_durations_dict[CalebSardinia20a_key] = CalebSardinia20a_durations.value.tolist()
L_obs_durations_dict[CalebNancay20a_key] = CalebNancay20a_durations.value.tolist()

#events per observation
L_events_per_obs_dict[Cruces_key] = Cruces_Eff_events.tolist()
L_events_per_obs_dict[Hardy_key] = Hardy_Eff_events.tolist()
L_events_per_obs_dict[Houben_key] = Houben_Eff_events.tolist()
L_events_per_obs_dict[ALFA_key] = ALFA_events.tolist()
L_events_per_obs_dict[Scholtz16_NH_key] = Scholtz16_NH_events.tolist()
L_events_per_obs_dict[Hewitt_key] = Hewitt_events.tolist()
L_events_per_obs_dict[Scholtz17_key] = Scholtz17_events.tolist()
L_events_per_obs_dict[Jahns_key] = Jahns_events.tolist()
L_events_per_obs_dict[Marcote_key] = Marcote_events.tolist()
L_events_per_obs_dict[MAGIC_key] = MAGIC_events.tolist()
L_events_per_obs_dict[Rajwade_key] = Rajwade_events.tolist()
L_events_per_obs_dict[Oostrum2017_key] = Oostrum2017_events.tolist()
L_events_per_obs_dict[CalebMeerKAT20a_key] = CalebMeerKAT20a_events.tolist()
L_events_per_obs_dict[CalebSardinia20a_key] = CalebSardinia20a_events.tolist()
L_events_per_obs_dict[CalebNancay20a_key] = CalebNancay20a_events.tolist()

In [35]:
print(L_bursts_dict.keys())

dict_keys(['Cruces21a', 'Hardy17a', 'Houben19a', 'ALFA16a', 'Scholtz16a', 'Hewitt22a', 'Scholtz17a', 'Jahns22a', 'Marcote16a', 'MAGIC17a', 'Rajwade20a', 'Oostrum17a', 'CalebMeerKAT20a', 'CalebSardinia20a', 'CalebNancay20a'])


In [36]:
#store to overall L-band dictionary

#initialise
L_dict = {}

#add data
L_dict['bursts'] = L_bursts_dict
L_dict['obs_duration'] = L_obs_durations_dict
L_dict['obs_startmjds'] = L_obs_startmjds_dict
L_dict['bursts_per_obs'] = L_events_per_obs_dict
L_dict['obs_tels'] = L_tels_dict
L_dict['obs_freqs'] = L_freqs_dict
L_dict['references'] = L_refs_dict

#save to json file format
with open("LBand_R1.json","w") as f:
    json.dump(L_dict,f)

In [37]:
print(L_dict.keys())

dict_keys(['bursts', 'obs_duration', 'obs_startmjds', 'bursts_per_obs', 'obs_tels', 'obs_freqs', 'references'])


### C-Band

In [38]:
#initialise data dictionaries
C_bursts_dict = {} #dictionary to hold L-band bursts
C_refs_dict = {} #dictionary to hold corresponding references
C_tels_dict = {} #dictionary to hold corresponding telescopes
C_freqs_dict = {} #dictionary to hold corresponding frequencies
C_obs_tels_dict = {} #dictionary to hold corresponding telescopes
C_obs_startmjds_dict = {} #dict to hold corresponding observation start mjds
C_obs_durations_dict = {} #dict to hold corresponding observation durations [s]
C_events_per_obs_dict = {} #dict to hold corresponding detections per individual obs

In [47]:
#add data

#burst mjds
C_bursts_dict[Michilli_key] = Michilli_mjds.tolist()
C_bursts_dict[Spitler_C_key] = Spitler_C_mjds.tolist()
#C_bursts_dict[Gaijar_key] = Gaijar_mjds.tolist() #gaijar bursts are in zhang paper
C_bursts_dict[Zhang_key] = Zhang_mjds.tolist()

C_bursts_dict[CalebEffelsberg20a_key] = CalebEffelsberg20a_mjds.tolist()


#references
C_refs_dict[Michilli_key] = Michilli_ref
C_refs_dict[Spitler_C_key] = Spitler_C_ref
#C_refs_dict[Gaijar_key] = Gaijar_ref #gaijar bursts are in zhang paper
C_refs_dict[Zhang_key] = Zhang_ref
C_refs_dict[CalebEffelsberg20a_key] = CalebEffelsberg20a_ref


#telescopes
C_tels_dict[Michilli_key] = Michilli_tel
C_tels_dict[Spitler_C_key] = Spitler_C_tel
#C_tels_dict[Gaijar_key] = Gaijar_tel #gaijar bursts are in zhang paper
C_tels_dict[Zhang_key] = Zhang_tel
C_tels_dict[CalebEffelsberg20a_key] = CalebEffelsberg20a_tel


#observing frequencies
C_freqs_dict[Michilli_key] = Michilli_freq
C_freqs_dict[Spitler_C_key] = Spitler_C_freq
#C_freqs_dict[Gaijar_key] = Gaijar_freq #gaijar bursts are in zhang paper
C_freqs_dict[Zhang_key] = Zhang_freq
C_freqs_dict[CalebEffelsberg20a_key] = CalebEffelsberg20a_freq


#start mjds
C_obs_startmjds_dict[Michilli_key] = Michilli_times.mjd.tolist()
C_obs_startmjds_dict[Spitler_C_key] = Spitler_C_times.mjd.tolist()
#C_obs_startmjds_dict[Gaijar_key] = Gaijar_times.mjd.tolist() #gaijar bursts are in zhang paper
C_obs_startmjds_dict[Zhang_key] = Zhang_times.mjd.tolist()
C_obs_startmjds_dict[CalebEffelsberg20a_key] = CalebEffelsberg20a_times.mjd.tolist()


#durations
C_obs_durations_dict[Michilli_key] = Michilli_durations.value.tolist()
C_obs_durations_dict[Spitler_C_key] = Spitler_C_durations.value.tolist()
#C_obs_durations_dict[Gaijar_key] = Gaijar_durations.value.tolist() #gaijar bursts are in zhang paper
C_obs_durations_dict[Zhang_key] = Zhang_durations.value.tolist()
C_obs_durations_dict[CalebEffelsberg20a_key] = CalebEffelsberg20a_durations.value.tolist()

#events per observation
C_events_per_obs_dict[Michilli_key] = Michilli_events.tolist()
C_events_per_obs_dict[Spitler_C_key] = Spitler_C_events.tolist()
#C_events_per_obs_dict[Gaijar_key] = Gaijar_events.tolist() #gaijar bursts are in zhang paper
C_events_per_obs_dict[Zhang_key] = Zhang_events.tolist()
C_events_per_obs_dict[CalebEffelsberg20a_key] = CalebEffelsberg20a_events.tolist()

In [48]:
print(C_bursts_dict.keys())

dict_keys(['Michilli18a', 'Spitler18a', 'Zhang18a', 'CalebEffelsberg20a'])


In [49]:
#store to overall C-band dictionary

#initialise
C_dict = {}

#add data
C_dict['bursts'] = C_bursts_dict
C_dict['obs_duration'] = C_obs_durations_dict
C_dict['obs_startmjds'] = C_obs_startmjds_dict
C_dict['bursts_per_obs'] = C_events_per_obs_dict
C_dict['obs_tels'] = C_tels_dict
C_dict['obs_freqs'] = C_freqs_dict
C_dict['references'] = C_refs_dict

#save to json file format
with open("CBand_R1.json","w") as f:
    json.dump(C_dict,f)

In [50]:
print(C_dict.keys())

dict_keys(['bursts', 'obs_duration', 'obs_startmjds', 'bursts_per_obs', 'obs_tels', 'obs_freqs', 'references'])


### Any Band

In [67]:
#initialise data dictionaries
AnyBand_bursts_dict = {} #dictionary to hold L-band bursts
AnyBand_refs_dict = {} #dictionary to hold corresponding references
AnyBand_tels_dict = {} #dictionary to hold corresponding telescopes
AnyBand_freqs_dict = {} #dictionary to hold corresponding frequencies
AnyBand_obs_tels_dict = {} #dictionary to hold corresponding telescopes
AnyBand_obs_startmjds_dict = {} #dict to hold corresponding observation start mjds
AnyBand_obs_durations_dict = {} #dict to hold corresponding observation durations [s]
AnyBand_events_per_obs_dict = {} #dict to hold corresponding detections per individual obs

In [68]:
#add data

#burst mjds
AnyBand_bursts_dict[Cruces_key] = Cruces_mjds.tolist()
AnyBand_bursts_dict[Hardy_key] = Hardy_mjds.tolist()
AnyBand_bursts_dict[Houben_key] = Houben_mjds.tolist()
AnyBand_bursts_dict[ALFA_key] = ALFA_mjds.tolist()
AnyBand_bursts_dict[Scholtz16_NH_key] = Scholtz16_NH_mjds.tolist()
AnyBand_bursts_dict[Hewitt_key] = Hewitt_mjds.tolist()
AnyBand_bursts_dict[Scholtz17_key] = Scholtz17_mjds.tolist()
AnyBand_bursts_dict[Jahns_key] = Jahns_mjds.tolist()
AnyBand_bursts_dict[Marcote_key] = Marcote_mjds.tolist()
AnyBand_bursts_dict[MAGIC_key] = MAGIC_mjds.tolist()
AnyBand_bursts_dict[Rajwade_key] = Rajwade_mjds.tolist()
AnyBand_bursts_dict[Oostrum2017_key] = Oostrum2017_mjds.tolist()
AnyBand_bursts_dict[CalebMeerKAT20a_key] = CalebMeerKAT20a_mjds.tolist()
AnyBand_bursts_dict[CalebSardinia20a_key] = CalebSardinia20a_mjds.tolist()
AnyBand_bursts_dict[CalebNancay20a_key] = CalebNancay20a_mjds.tolist()
AnyBand_bursts_dict[Michilli_key] = Michilli_mjds.tolist()
AnyBand_bursts_dict[Spitler_C_key] = Spitler_C_mjds.tolist()
#AnyBand_bursts_dict[Gaijar_key] = Gaijar_mjds.tolist() #Gaijar bursts are in Zhang paper
AnyBand_bursts_dict[Zhang_key] = Zhang_mjds.tolist()
AnyBand_bursts_dict[CalebEffelsberg20a_key] = CalebEffelsberg20a_mjds.tolist()

#references
AnyBand_refs_dict[Cruces_key] = Cruces_ref
AnyBand_refs_dict[Hardy_key] = Hardy_ref
AnyBand_refs_dict[Houben_key] = Houben_ref
AnyBand_refs_dict[ALFA_key] = ALFA_ref
AnyBand_refs_dict[Scholtz16_NH_key] = Scholtz16_NH_ref
AnyBand_refs_dict[Hewitt_key] = Hewitt_ref
AnyBand_refs_dict[Scholtz17_key] = Scholtz17_ref
AnyBand_refs_dict[Jahns_key] = Jahns_ref
AnyBand_refs_dict[Marcote_key] = Marcote_ref
AnyBand_refs_dict[MAGIC_key] = MAGIC_ref
AnyBand_refs_dict[Rajwade_key] = Rajwade_ref
AnyBand_refs_dict[Oostrum2017_key] = Oostrum2017_ref
AnyBand_refs_dict[CalebMeerKAT20a_key] = CalebMeerKAT20a_ref
AnyBand_refs_dict[CalebSardinia20a_key] = CalebSardinia20a_ref
AnyBand_refs_dict[CalebNancay20a_key] = CalebNancay20a_ref
AnyBand_refs_dict[Michilli_key] = Michilli_ref
AnyBand_refs_dict[Spitler_C_key] = Spitler_C_ref
#AnyBand_refs_dict[Gaijar_key] = Gaijar_ref #Gaijar bursts are in Zhang paper
AnyBand_refs_dict[Zhang_key] = Zhang_ref
AnyBand_refs_dict[CalebEffelsberg20a_key] = CalebEffelsberg20a_ref


#telescopes
AnyBand_tels_dict[Cruces_key] = Cruces_tel
AnyBand_tels_dict[Hardy_key] = Hardy_tel
AnyBand_tels_dict[Houben_key] = Houben_tel
AnyBand_tels_dict[ALFA_key] = ALFA_tel
AnyBand_tels_dict[Scholtz16_NH_key] = Scholtz16_NH_tel
AnyBand_tels_dict[Hewitt_key] = Hewitt_tel
AnyBand_tels_dict[Scholtz17_key] = Scholtz17_tel
AnyBand_tels_dict[Jahns_key] = Jahns_tel
AnyBand_tels_dict[Marcote_key] = Marcote_tel
AnyBand_tels_dict[MAGIC_key] = MAGIC_tel
AnyBand_tels_dict[Rajwade_key] = Rajwade_tel
AnyBand_tels_dict[Oostrum2017_key] = Oostrum2017_tel
AnyBand_tels_dict[CalebMeerKAT20a_key] = CalebMeerKAT20a_tel
AnyBand_tels_dict[CalebSardinia20a_key] = CalebSardinia20a_tel
AnyBand_tels_dict[CalebNancay20a_key] = CalebNancay20a_tel
AnyBand_tels_dict[Michilli_key] = Michilli_tel
AnyBand_tels_dict[Spitler_C_key] = Spitler_C_tel
#AnyBand_tels_dict[Gaijar_key] = Gaijar_tel #Gaijar bursts are in Zhang paper
AnyBand_tels_dict[Zhang_key] = Zhang_tel
AnyBand_tels_dict[CalebEffelsberg20a_key] = CalebEffelsberg20a_tel

#observing frequencies
AnyBand_freqs_dict[Cruces_key] = Cruces_freq
AnyBand_freqs_dict[Hardy_key] = Hardy_freq
AnyBand_freqs_dict[Houben_key] = Houben_freq
AnyBand_freqs_dict[ALFA_key] = ALFA_freq
AnyBand_freqs_dict[Scholtz16_NH_key] = Scholtz16_NH_freq
AnyBand_freqs_dict[Hewitt_key] = Hewitt_freq
AnyBand_freqs_dict[Scholtz17_key] = Scholtz17_freq
AnyBand_freqs_dict[Jahns_key] = Jahns_freq
AnyBand_freqs_dict[Marcote_key] = Marcote_freq
AnyBand_freqs_dict[MAGIC_key] = MAGIC_freq
AnyBand_freqs_dict[Rajwade_key] = Rajwade_freq
AnyBand_freqs_dict[Oostrum2017_key] = Oostrum2017_freq
AnyBand_freqs_dict[CalebMeerKAT20a_key] = CalebMeerKAT20a_freq
AnyBand_freqs_dict[CalebSardinia20a_key] = CalebSardinia20a_freq
AnyBand_freqs_dict[CalebNancay20a_key] = CalebNancay20a_freq
AnyBand_freqs_dict[Michilli_key] = Michilli_freq
AnyBand_freqs_dict[Spitler_C_key] = Spitler_C_freq
#AnyBand_freqs_dict[Gaijar_key] = Gaijar_freq #Gaijar bursts are in Zhang paper
AnyBand_freqs_dict[Zhang_key] = Zhang_freq
AnyBand_freqs_dict[CalebEffelsberg20a_key] = CalebEffelsberg20a_freq

#start mjds
AnyBand_obs_startmjds_dict[Cruces_key] = Cruces_Eff_times.mjd.tolist()
AnyBand_obs_startmjds_dict[Hardy_key] = Hardy_Eff_times.mjd.tolist()
AnyBand_obs_startmjds_dict[Houben_key] = Houben_Eff_times.mjd.tolist()
AnyBand_obs_startmjds_dict[ALFA_key] = ALFA_times.mjd.tolist()
AnyBand_obs_startmjds_dict[Scholtz16_NH_key] = Scholtz16_NH_times.mjd.tolist()
AnyBand_obs_startmjds_dict[Hewitt_key] = Hewitt_times.mjd.tolist()
AnyBand_obs_startmjds_dict[Scholtz17_key] = Scholtz17_times.mjd.tolist()
AnyBand_obs_startmjds_dict[Jahns_key] = Jahns_times.mjd.tolist()
AnyBand_obs_startmjds_dict[Marcote_key] = Marcote_times.mjd.tolist()
AnyBand_obs_startmjds_dict[MAGIC_key] = MAGIC_times.mjd.tolist()
AnyBand_obs_startmjds_dict[Rajwade_key] = Rajwade_times.mjd.tolist()
AnyBand_obs_startmjds_dict[Oostrum2017_key] = Oostrum2017_times.mjd.tolist()
AnyBand_obs_startmjds_dict[CalebMeerKAT20a_key] = CalebMeerKAT20a_times.mjd.tolist()
AnyBand_obs_startmjds_dict[CalebSardinia20a_key] = CalebSardinia20a_times.mjd.tolist()
AnyBand_obs_startmjds_dict[CalebNancay20a_key] = CalebNancay20a_times.mjd.tolist()
AnyBand_obs_startmjds_dict[Michilli_key] = Michilli_times.mjd.tolist()
AnyBand_obs_startmjds_dict[Spitler_C_key] = Spitler_C_times.mjd.tolist()
#AnyBand_obs_startmjds_dict[Gaijar_key] = Gaijar_times.mjd.tolist() #Gaijar bursts are in Zhang paper
AnyBand_obs_startmjds_dict[Zhang_key] = Zhang_times.mjd.tolist()
AnyBand_obs_startmjds_dict[CalebEffelsberg20a_key] = CalebEffelsberg20a_times.mjd.tolist()

#durations
AnyBand_obs_durations_dict[Cruces_key] = Cruces_Eff_durations.value.tolist()
AnyBand_obs_durations_dict[Hardy_key] = Hardy_Eff_durations.value.tolist()
AnyBand_obs_durations_dict[Houben_key] = Houben_Eff_durations.value.tolist()
AnyBand_obs_durations_dict[ALFA_key] = ALFA_durations.value.tolist()
AnyBand_obs_durations_dict[Scholtz16_NH_key] = Scholtz16_NH_durations.value.tolist()
AnyBand_obs_durations_dict[Hewitt_key] = Hewitt_durations.value.tolist()
AnyBand_obs_durations_dict[Scholtz17_key] = Scholtz17_durations.value.tolist()
AnyBand_obs_durations_dict[Jahns_key] = Jahns_durations.value.tolist()
AnyBand_obs_durations_dict[Marcote_key] = Marcote_durations.value.tolist()
AnyBand_obs_durations_dict[MAGIC_key] = MAGIC_durations.value.tolist()
AnyBand_obs_durations_dict[Rajwade_key] = Rajwade_durations.value.tolist()
AnyBand_obs_durations_dict[Oostrum2017_key] = Oostrum2017_durations.value.tolist()
AnyBand_obs_durations_dict[CalebMeerKAT20a_key] = CalebMeerKAT20a_durations.value.tolist()
AnyBand_obs_durations_dict[CalebSardinia20a_key] = CalebSardinia20a_durations.value.tolist()
AnyBand_obs_durations_dict[CalebNancay20a_key] = CalebNancay20a_durations.value.tolist()
AnyBand_obs_durations_dict[Michilli_key] = Michilli_durations.value.tolist()
AnyBand_obs_durations_dict[Spitler_C_key] = Spitler_C_durations.value.tolist()
#AnyBand_obs_durations_dict[Gaijar_key] = Gaijar_durations.value.tolist() #Gaijar bursts are in Zhang paper
AnyBand_obs_durations_dict[Zhang_key] = Zhang_durations.value.tolist()
AnyBand_obs_durations_dict[CalebEffelsberg20a_key] = CalebEffelsberg20a_durations.value.tolist()

#events per observation
AnyBand_events_per_obs_dict[Cruces_key] = Cruces_Eff_events.tolist()
AnyBand_events_per_obs_dict[Hardy_key] = Hardy_Eff_events.tolist()
AnyBand_events_per_obs_dict[Houben_key] = Houben_Eff_events.tolist()
AnyBand_events_per_obs_dict[ALFA_key] = ALFA_events.tolist()
AnyBand_events_per_obs_dict[Scholtz16_NH_key] = Scholtz16_NH_events.tolist()
AnyBand_events_per_obs_dict[Hewitt_key] = Hewitt_events.tolist()
AnyBand_events_per_obs_dict[Scholtz17_key] = Scholtz17_events.tolist()
AnyBand_events_per_obs_dict[Jahns_key] = Jahns_events.tolist()
AnyBand_events_per_obs_dict[Marcote_key] = Marcote_events.tolist()
AnyBand_events_per_obs_dict[MAGIC_key] = MAGIC_events.tolist()
AnyBand_events_per_obs_dict[Rajwade_key] = Rajwade_events.tolist()
AnyBand_events_per_obs_dict[Oostrum2017_key] = Oostrum2017_events.tolist()
AnyBand_events_per_obs_dict[CalebMeerKAT20a_key] = CalebMeerKAT20a_events.tolist()
AnyBand_events_per_obs_dict[CalebSardinia20a_key] = CalebSardinia20a_events.tolist()
AnyBand_events_per_obs_dict[CalebNancay20a_key] = CalebNancay20a_events.tolist()
AnyBand_events_per_obs_dict[Michilli_key] = Michilli_events.tolist()
AnyBand_events_per_obs_dict[Spitler_C_key] = Spitler_C_events.tolist()
#AnyBand_events_per_obs_dict[Gaijar_key] = Gaijar_events.tolist() #Gaijar bursts are in Zhang paper
AnyBand_events_per_obs_dict[Zhang_key] = Zhang_events.tolist()
AnyBand_events_per_obs_dict[CalebEffelsberg20a_key] = CalebEffelsberg20a_events.tolist()




In [69]:
print(AnyBand_bursts_dict.keys())

dict_keys(['Cruces21a', 'Hardy17a', 'Houben19a', 'ALFA16a', 'Scholtz16a', 'Hewitt22a', 'Scholtz17a', 'Jahns22a', 'Marcote16a', 'MAGIC17a', 'Rajwade20a', 'Oostrum17a', 'CalebMeerKAT20a', 'CalebSardinia20a', 'CalebNancay20a', 'Michilli18a', 'Spitler18a', 'Zhang18a', 'CalebEffelsberg20a'])


In [70]:
#store to overall Any-Band dictionary

#initialise
AnyBand_dict = {}

#add data
AnyBand_dict['bursts'] = AnyBand_bursts_dict
AnyBand_dict['obs_duration'] = AnyBand_obs_durations_dict
AnyBand_dict['obs_startmjds'] = AnyBand_obs_startmjds_dict
AnyBand_dict['bursts_per_obs'] = AnyBand_events_per_obs_dict
AnyBand_dict['obs_tels'] = AnyBand_tels_dict
AnyBand_dict['obs_freqs'] = AnyBand_freqs_dict
AnyBand_dict['references'] = AnyBand_refs_dict

#save to json file format
with open("AnyBand_R1.json","w") as f:
    json.dump(AnyBand_dict,f)

In [71]:
print(AnyBand_dict.keys())

dict_keys(['bursts', 'obs_duration', 'obs_startmjds', 'bursts_per_obs', 'obs_tels', 'obs_freqs', 'references'])
